# RTpipeline on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kstawiski/rtpipeline/blob/main/rtpipeline_colab.ipynb)

This notebook allows you to run the RTpipeline radiotherapy data processing system on Google Colab with GPU acceleration.

## What This Pipeline Does

RTpipeline processes DICOM radiotherapy data and generates:
- ✅ **Automatic segmentation** of 100+ organs using TotalSegmentator
- ✅ **DVH metrics** (dose-volume histograms)
- ✅ **Radiomics features** (150+ texture/shape features)
- ✅ **Radiomics robustness testing** (ICC/CoV stability analysis) - **NEW!**
- ✅ **Quality control reports**
- ✅ **Analysis-ready tables** for machine learning

## Prerequisites

- Google Colab account (free tier works, but GPU runtime recommended)
- DICOM files (CT, RTPLAN, RTDOSE, RTSTRUCT)
- ~10-30 minutes processing time per patient (GPU)
- +10-30 minutes if robustness testing enabled

---

**⚡ Quick Start:** Run all cells in order

## 1️⃣ Setup: Install Miniconda

This cell installs Miniconda for managing conda environments (~2 minutes)

In [ ]:
%%bash
# Check GPU availability
echo "=== GPU Check ==="
nvidia-smi || echo "⚠️ No GPU detected. Pipeline will use CPU (slower)."

# Install system dependencies
echo -e "\n=== Installing System Dependencies ==="
apt-get update -qq
apt-get install -y -qq dcm2niix pigz > /dev/null

echo -e "\n=== Installing Python dependencies (pydicom, SimpleITK, etc.) ==="
python3 -m pip install -q "pydicom>=3.0.0" "SimpleITK>=2.3.0" "dicompyler-core>=0.5.6" "rt-utils>=1.4.0" "nibabel>=5.1.0" "xlsxwriter" "openpyxl"
echo "✅ Core Python deps installed"

# Install Miniconda if not already installed
if [ ! -d "/content/miniconda" ]; then
    echo -e "\n=== Installing Miniconda ==="
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /tmp/miniconda.sh
    bash /tmp/miniconda.sh -b -p /content/miniconda
    rm /tmp/miniconda.sh
    echo "✅ Miniconda installed"
else
    echo -e "\n=== Miniconda already installed ==="
fi

# Initialize conda
export PATH="/content/miniconda/bin:$PATH"
eval "$(/content/miniconda/bin/conda shell.bash hook)"
conda init bash


echo -e "\n=== Installing Snakemake (base env) ==="
conda install -n base -c conda-forge -c bioconda -y -q snakemake
echo -e "\n=== Conda initialized ==="
echo "Conda version: $(conda --version)"

echo -e "\n✅ Setup complete!"

## 1️⃣b Setup: Create Conda Environments

This cell creates two separate conda environments to handle numpy version conflicts:
- **rtpipeline**: TotalSegmentator (requires numpy>=2.0)
- **rtpipeline-radiomics**: PyRadiomics (requires numpy=1.26.x)

This takes ~5-10 minutes but only needs to be done once per session.

In [ ]:
%%bash
export PATH="/content/miniconda/bin:$PATH"
eval "$(/content/miniconda/bin/conda shell.bash hook)"

# Accept Anaconda Terms of Service for required channels
echo "=== Accepting Anaconda Terms of Service ==="
# NOTE: The project's README.md (lines 397 and 627) recommends 'channel_priority strict'
# to avoid environment inconsistencies. However, in Google Colab, we must use 'flexible'
# to allow ToS acceptance and compatibility with Colab's pre-installed packages.
# This may lead to package version conflicts, but is required for Colab to work.
conda config --set channel_priority flexible
if ! conda tos accept --channel defaults 2>&1; then
    echo "⚠️ ToS acceptance for defaults channel failed or already accepted"
fi
echo "✅ ToS accepted"
echo ""

# Check if repository is already cloned (needed for env files)
if [ ! -d "/content/rtpipeline" ]; then
    echo "Cloning rtpipeline repository (needed for environment files)..."
    git clone -q https://github.com/kstawiski/rtpipeline.git /content/rtpipeline
    echo "✅ Repository cloned"
fi

cd /content/rtpipeline

echo "=== Creating Conda Environments ==="
echo ""

# Create rtpipeline environment (for TotalSegmentator with numpy>=2.0)
if conda env list | grep -q "^rtpipeline "; then
    echo "✅ Environment 'rtpipeline' already exists"
else
    echo "Creating 'rtpipeline' environment (TotalSegmentator, numpy>=2.0)..."
    conda env create -f envs/rtpipeline.yaml -q
    echo "✅ Environment 'rtpipeline' created"
fi

echo ""

# Create rtpipeline-radiomics environment (for pyradiomics with numpy=1.26.x)
if conda env list | grep -q "^rtpipeline-radiomics "; then
    echo "✅ Environment 'rtpipeline-radiomics' already exists"
else
    echo "Creating 'rtpipeline-radiomics' environment (PyRadiomics, numpy=1.26.x)..."
    conda env create -f envs/rtpipeline-radiomics.yaml -q
    echo "✅ Environment 'rtpipeline-radiomics' created"
fi

echo ""
echo "=== Environment Setup Complete ==="
echo ""
echo "Available conda environments:"
conda env list

echo ""
echo "Verifying numpy versions:"
echo "- rtpipeline:"
conda run -n rtpipeline python -c "import numpy; print(f'  numpy {numpy.__version__}')"
echo "- rtpipeline-radiomics:"
conda run -n rtpipeline-radiomics python -c "import numpy; print(f'  numpy {numpy.__version__}')"

## 2️⃣ Clone RTpipeline Repository

In [ ]:
%%bash
# Clone repository
if [ ! -d "/content/rtpipeline" ]; then
    echo "Cloning rtpipeline repository..."
    git clone https://github.com/kstawiski/rtpipeline.git /content/rtpipeline
    echo "✅ Repository cloned"
else
    echo "✅ Repository already exists"
fi

cd /content/rtpipeline
git pull origin main
echo "Repository updated to latest version"

## 3️⃣ Upload Your DICOM Files

You have two options:

### Option A: Upload from Google Drive

Run this cell to mount your Google Drive, then access files from `/content/drive/MyDrive/`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted at /content/drive/MyDrive/")
print("\nYour DICOM files should be in: /content/drive/MyDrive/your_dicom_folder/")

### Option B: Upload Files Directly

Use the file browser on the left (📁 icon) to upload DICOM files to `/content/dicom_data/`

In [ ]:
%%bash
# Create directories
mkdir -p /content/dicom_data
mkdir -p /content/output
mkdir -p /content/logs

echo "✅ Directories created:"
echo "  - /content/dicom_data (upload your DICOM files here)"
echo "  - /content/output (results will be saved here)"
echo "  - /content/logs (processing logs)"

## 4️⃣ Configure Pipeline

Modify the settings below according to your needs:

In [ ]:
import os

# ============ CONFIGURATION ============

# DICOM directory (change if using Google Drive)
DICOM_ROOT = "/content/dicom_data"
# Example: DICOM_ROOT = "/content/drive/MyDrive/my_dicom_folder"

# Output directory (write directly to Drive to avoid /content overflow)
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/rtpipeline_output"
OUTPUT_DIR = DRIVE_OUTPUT_DIR
LOGS_DIR = "/content/logs"  # Keep logs local for faster writes
LOCAL_TEMP_DIR = "/content/tmp_rtpipeline"
SEG_TEMP_DIR = os.path.join(LOCAL_TEMP_DIR, "totalseg")
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
os.makedirs(SEG_TEMP_DIR, exist_ok=True)

# Processing options
USE_GPU = True  # Set to False if no GPU available
ENABLE_RADIOMICS = True  # Extract radiomic features
ENABLE_CT_CROPPING = False  # Crop CT to anatomical region
CROP_REGION = "pelvis"  # Options: pelvis, thorax, abdomen, head_neck, brain

# Advanced parallelization settings
# NOTE: Colab uses cloud/network storage which is I/O-bound!
CPU_COUNT = os.cpu_count() or 2
WORKERS = max(1, CPU_COUNT - 1)  # I/O parallelism per course
SNAKEMAKE_JOB_THREADS = WORKERS
SEG_WORKERS = 1  # GPU-limited: keep sequential
FAST_MODE = False  # CPU-friendly mode (lower quality)

# =======================================


### 🧪 Radiomics Robustness Testing (Optional)

**NEW**: Test feature stability under clinical variations!

Radiomics robustness testing evaluates whether your radiomic features remain stable under:
- **Volume changes** (±15-30%): Simulates segmentation uncertainty
- **Translations** (±3-5mm): Simulates positioning variability
- **Noise injection** (10-20 HU): Simulates scanner differences
- **Contour randomization**: Simulates inter-observer variability

**Outputs:**
- ICC (Intraclass Correlation) ≥0.90 = robust features
- CoV (Coefficient of Variation) ≤10% = stable features
- Excel report with robust/acceptable/poor feature classifications

⏱️ **Time:** Adds 10-30 minutes per patient depending on intensity

In [ ]:
# ============ ROBUSTNESS TESTING CONFIGURATION ============

# Enable robustness testing?
ENABLE_ROBUSTNESS = False  # Set to True to enable

# Structures to test (use wildcards: "GTV*", "CTV*")
ROBUSTNESS_STRUCTURES = [
    "GTV*",
    "CTV*", 
    "PTV*",
    "urinary_bladder",
    "rectum",
    "prostate"
]

# Intensity level: "mild", "standard", "aggressive"
# - mild: 10-15 perturbations per ROI (~5-10 min/patient with GPU)
# - standard: 15-30 perturbations (~10-20 min/patient) [RECOMMENDED]
# - aggressive: 30-60 perturbations (~30-60 min/patient, research-grade)
ROBUSTNESS_INTENSITY = "standard"

# Perturbation types (enable by setting values):
ROBUSTNESS_VOLUME_CHANGES = [-0.15, 0.0, 0.15]  # ±15% volume changes
ROBUSTNESS_TRANSLATIONS_MM = 0.0  # Set to 3.0 or 5.0 to enable translations
ROBUSTNESS_NOISE_LEVELS = [0.0]  # Set to [0.0, 10.0, 20.0] to enable noise
ROBUSTNESS_CONTOUR_REALIZATIONS = 0  # Set to 2-3 to enable contour randomization

# Advanced: Large volume changes for organs (optional)
ROBUSTNESS_LARGE_VOLUME_CHANGES = [-0.30, 0.0, 0.30]  # ±30% for OARs

# ===========================================================

print("Robustness Testing Configuration:")
print(f"  Enabled: {ENABLE_ROBUSTNESS}")
if ENABLE_ROBUSTNESS:
    print(f"  Intensity: {ROBUSTNESS_INTENSITY}")
    print(f"  Structures: {len(ROBUSTNESS_STRUCTURES)} patterns")
    print(f"  Volume changes: {ROBUSTNESS_VOLUME_CHANGES}")
    print(f"  Translations: {ROBUSTNESS_TRANSLATIONS_MM} mm")
    print(f"  Noise levels: {ROBUSTNESS_NOISE_LEVELS} HU")
    print(f"  Contour realizations: {ROBUSTNESS_CONTOUR_REALIZATIONS}")
else:
    print("  (Set ENABLE_ROBUSTNESS = True to activate)")

### 🧪 Radiomics Robustness Testing (Optional)

**NEW**: Test feature stability under clinical variations!

Radiomics robustness testing evaluates whether your radiomic features remain stable under:
- **Volume changes** (±15-30%): Simulates segmentation uncertainty
- **Translations** (±3-5mm): Simulates positioning variability
- **Noise injection** (10-20 HU): Simulates scanner differences
- **Contour randomization**: Simulates inter-observer variability

**Outputs:**
- ICC (Intraclass Correlation) ≥0.90 = robust features
- CoV (Coefficient of Variation) ≤10% = stable features
- Excel report with robust/acceptable/poor feature classifications

⏱️ **Time:** Adds 10-30 minutes per patient depending on intensity

In [ ]:
# ============ ROBUSTNESS TESTING CONFIGURATION ============

# Enable robustness testing?
ENABLE_ROBUSTNESS = False  # Set to True to enable

# Structures to test (use wildcards: "GTV*", "CTV*")
ROBUSTNESS_STRUCTURES = [
    "GTV*",
    "CTV*", 
    "PTV*",
    "urinary_bladder",
    "rectum",
    "prostate"
]

# Intensity level: "mild", "standard", "aggressive"
# - mild: 10-15 perturbations per ROI (~5-10 min/patient with GPU)
# - standard: 15-30 perturbations (~10-20 min/patient) [RECOMMENDED]
# - aggressive: 30-60 perturbations (~30-60 min/patient, research-grade)
ROBUSTNESS_INTENSITY = "standard"

# Perturbation types (enable by setting values):
ROBUSTNESS_VOLUME_CHANGES = [-0.15, 0.0, 0.15]  # ±15% volume changes
ROBUSTNESS_TRANSLATIONS_MM = 0.0  # Set to 3.0 or 5.0 to enable translations
ROBUSTNESS_NOISE_LEVELS = [0.0]  # Set to [0.0, 10.0, 20.0] to enable noise
ROBUSTNESS_CONTOUR_REALIZATIONS = 0  # Set to 2-3 to enable contour randomization

# Advanced: Large volume changes for organs (optional)
ROBUSTNESS_LARGE_VOLUME_CHANGES = [-0.30, 0.0, 0.30]  # ±30% for OARs

# ===========================================================

print("Robustness Testing Configuration:")
print(f"  Enabled: {ENABLE_ROBUSTNESS}")
if ENABLE_ROBUSTNESS:
    print(f"  Intensity: {ROBUSTNESS_INTENSITY}")
    print(f"  Structures: {len(ROBUSTNESS_STRUCTURES)} patterns")
    print(f"  Volume changes: {ROBUSTNESS_VOLUME_CHANGES}")
    print(f"  Translations: {ROBUSTNESS_TRANSLATIONS_MM} mm")
    print(f"  Noise levels: {ROBUSTNESS_NOISE_LEVELS} HU")
    print(f"  Contour realizations: {ROBUSTNESS_CONTOUR_REALIZATIONS}")
else:
    print("  (Set ENABLE_ROBUSTNESS = True to activate)")

## 5️⃣ Generate Configuration File

In [ ]:
config_yaml = f"""# RTpipeline Configuration for Google Colab
# Generated automatically

# Input/Output directories
dicom_root: "{DICOM_ROOT}"
output_dir: "{OUTPUT_DIR}"
logs_dir: "{LOGS_DIR}"

# Processing parameters
snakemake_job_threads: {WORKERS}
workers: {WORKERS}

segmentation:
  workers: {SEG_WORKERS}
  threads_per_worker: null
  force: false
  fast: {str(FAST_MODE).lower()}
  roi_subset: null
  extra_models: []
  device: "{'gpu' if USE_GPU else 'cpu'}"
  force_split: true
  nr_threads_resample: 1
  nr_threads_save: 1
  num_proc_preprocessing: 1
  num_proc_export: 1
  temp_dir: "{SEG_TEMP_DIR}"

custom_models:
  enabled: false
  root: "custom_models"
  models: []
  workers: 1
  force: false
  nnunet_predict: "nnUNet_predict"
  retain_weights: true
  conda_activate: null

radiomics:
  sequential: false
  params_file: "/content/rtpipeline/rtpipeline/radiomics_params.yaml"
  mr_params_file: "/content/rtpipeline/rtpipeline/radiomics_params_mr.yaml"
  thread_limit: 4
  skip_rois:
    - body
    - couchsurface
    - bones
  max_voxels: 1500000000
  min_voxels: 10

aggregation:
  threads: auto

environments:
  main: "base"
  radiomics: "base"

custom_structures: "custom_structures_pelvic.yaml"

ct_cropping:
  enabled: {str(ENABLE_CT_CROPPING).lower()}
  region: "{CROP_REGION}"
  superior_margin_cm: 2.0
  inferior_margin_cm: 10.0
  use_cropped_for_dvh: true
  use_cropped_for_radiomics: true
  keep_original: true
"""
config_path = "/content/config_colab.yaml"

with open(config_path, "w") as f:
    f.write(config_yaml)

print(f"✅ Configuration saved to {config_path}")
print("Review with: !cat /content/config_colab.yaml")


## 6️⃣ Run Pipeline with Snakemake

This uses Snakemake to orchestrate the pipeline with proper conda environment management:
- **TotalSegmentator** runs in `rtpipeline` environment (numpy>=2.0)
- **PyRadiomics** runs in `rtpipeline-radiomics` environment (numpy=1.26.x)

⏱️ **Estimated time:**
- With GPU: 10-30 minutes per patient
- Without GPU: 1-3 hours per patient

**Note:** Colab may timeout after 12 hours. For large datasets, process in batches.

## 6️⃣b Run Radiomics Robustness Testing (Optional)

Run this cell if you enabled robustness testing (`ENABLE_ROBUSTNESS = True`)

This will:
1. Apply perturbations to segmentations (volume, translation, noise, contour)
2. Extract radiomic features for each perturbation
3. Compute ICC and CoV stability metrics
4. Generate Excel report with robust/acceptable/poor feature classifications

In [ ]:
if ENABLE_ROBUSTNESS:
    import subprocess
    import os
    import glob
    
    os.chdir('/content/rtpipeline')
    
    # Setup conda environment
    os.environ['PATH'] = f"/content/miniconda/bin:{os.environ.get('PATH', '')}"
    
    print("=== Starting Radiomics Robustness Testing ===")
    print(f"Configuration: /content/config_colab.yaml")
    print(f"Structures: {ROBUSTNESS_STRUCTURES}")
    print(f"Intensity: {ROBUSTNESS_INTENSITY}")
    print("Using conda environment: rtpipeline-radiomics (numpy=1.26.x)")
    print("")
    
    # Find all course directories
    course_dirs = glob.glob(f"{OUTPUT_DIR}/Data_Snakemake/*/*")
    course_dirs = [d for d in course_dirs if os.path.isdir(d) and not d.endswith('_RESULTS') and not d.startswith('_')]
    
    if not course_dirs:
        print("⚠️ No course directories found. Make sure the main pipeline completed successfully.")
    else:
        print(f"Found {len(course_dirs)} course(s) to process")
        print("")
        
        # Process each course
        robustness_files = []
        for i, course_dir in enumerate(course_dirs, 1):
            course_name = os.path.basename(course_dir)
            patient_name = os.path.basename(os.path.dirname(course_dir))
            
            print(f"[{i}/{len(course_dirs)}] Processing: {patient_name}/{course_name}")
            
            output_file = os.path.join(course_dir, "radiomics_robustness_ct.parquet")
            
            # Use conda run to execute in the radiomics environment
            cmd = [
                "conda", "run", "-n", "rtpipeline-radiomics",
                "python3", "-m", "rtpipeline.cli",
                "radiomics-robustness",
                "--course-dir", course_dir,
                "--config", "/content/config_colab.yaml",
                "--output", output_file
            ]
            
            try:
                result = subprocess.run(cmd, check=True, capture_output=True, text=True)
                print(f"  ✅ Complete: {output_file}")
                robustness_files.append(output_file)
            except subprocess.CalledProcessError as e:
                print(f"  ⚠️ Failed: {e}")
                print(f"  Error: {e.stderr[:500]}")
            
            print("")
        
        # Aggregate results
        if robustness_files:
            print("=== Aggregating Robustness Results ===")
            
            output_excel = f"{OUTPUT_DIR}/_RESULTS/radiomics_robustness_summary.xlsx"
            
            cmd = [
                "conda", "run", "-n", "rtpipeline-radiomics",
                "python3", "-m", "rtpipeline.cli",
                "radiomics-robustness-aggregate",
                "--config", "/content/config_colab.yaml",
                "--output", output_excel,
                "--inputs"
            ] + robustness_files
            
            try:
                result = subprocess.run(cmd, check=True, capture_output=True, text=True)
                print(f"✅ Aggregated summary saved to: {output_excel}")
                print("")
                print("=== Robustness Testing Complete ===")
                print(f"Results: {output_excel}")
            except subprocess.CalledProcessError as e:
                print(f"⚠️ Aggregation failed: {e}")
                print(f"Error: {e.stderr[:500]}")
        else:
            print("⚠️ No robustness results to aggregate")
else:
    print("⚠️ Robustness testing disabled. Set ENABLE_ROBUSTNESS = True to enable.")

In [ ]:
import os
import subprocess

# Export Python variables as environment variables for the shell
os.environ['WORKERS'] = str(WORKERS)
os.environ['OUTPUT_DIR'] = OUTPUT_DIR

# Setup conda path
os.environ['PATH'] = f"/content/miniconda/bin:{os.environ.get('PATH', '')}"

# Change to rtpipeline directory
os.chdir('/content/rtpipeline')

print("=== Starting RTpipeline with Snakemake ===")
print("Configuration: /content/config_colab.yaml")
print("")
print("This will use separate conda environments:")
print("  - rtpipeline: for TotalSegmentator (numpy>=2.0)")
print("  - rtpipeline-radiomics: for PyRadiomics (numpy=1.26.x)")
print("")

# Install Snakemake if needed
try:
    subprocess.run(
        ["conda", "run", "-n", "base", "snakemake", "--version"],
        check=True,
        capture_output=True
    )
except subprocess.CalledProcessError:
    print("Installing Snakemake...")
    subprocess.run(
        ["conda", "install", "-n", "base", "-c", "conda-forge", "-c", "bioconda", "snakemake", "-y", "-q"],
        check=True
    )
    print("✅ Snakemake installed")

# Run Snakemake
cmd = [
    "conda", "run", "-n", "base", "snakemake",
    "--configfile", "/content/config_colab.yaml",
    "--use-conda",
    "--cores", str(WORKERS),
    "--printshellcmds",
    "--keep-going"
]

result = subprocess.run(cmd, capture_output=False, text=True)

print("")
print("=== Pipeline Complete ===")
print(f"Results saved to: {OUTPUT_DIR}")
print(f"Check aggregated results: {OUTPUT_DIR}/_RESULTS/")


## 7️⃣ View Results

Load and preview the aggregated results:

In [ ]:
import pandas as pd
import os

results_dir = f"{OUTPUT_DIR}/_RESULTS"

# Check if results exist
if not os.path.exists(results_dir):
    print("⚠️ Results directory not found. Pipeline may still be running or failed.")
    print(f"Expected location: {results_dir}")
else:
    print("✅ Results found!\n")

    # List result files
    result_files = os.listdir(results_dir)
    print("Available files:")
    for f in result_files:
        if f.endswith('.xlsx'):
            filepath = os.path.join(results_dir, f)
            size_mb = os.path.getsize(filepath) / 1024 / 1024
            print(f"  - {f} ({size_mb:.2f} MB)")

    # Load DVH metrics
    try:
        dvh_path = os.path.join(results_dir, "dvh_metrics.xlsx")
        dvh = pd.read_excel(dvh_path)
        print(f"\n✅ Loaded DVH metrics: {len(dvh)} rows")
        print("\nFirst few rows:")
        display(dvh.head())

        print("\nStructures found:")
        print(dvh['Structure'].value_counts().head(10))
    except Exception as e:
        print(f"\n⚠️ Could not load DVH metrics: {e}")

    # Load radiomics
    if ENABLE_RADIOMICS:
        try:
            rad_path = os.path.join(results_dir, "radiomics_ct.xlsx")
            radiomics = pd.read_excel(rad_path)
            print(f"\n✅ Loaded radiomics: {len(radiomics)} rows, {len(radiomics.columns)} features")
            print("\nFirst few rows:")
            display(radiomics.head())
        except Exception as e:
            print(f"\n⚠️ Could not load radiomics: {e}")

    # Load robustness results
    if ENABLE_ROBUSTNESS:
        try:
            rob_path = os.path.join(results_dir, "radiomics_robustness_summary.xlsx")
            
            # Load different sheets
            robust_summary = pd.read_excel(rob_path, sheet_name='global_summary')
            robust_features = pd.read_excel(rob_path, sheet_name='robust_features')
            
            print(f"\n✅ Loaded robustness summary: {len(robust_summary)} features analyzed")
            print(f"   Robust features (ICC ≥0.90, CoV ≤10%): {len(robust_features)}")
            
            print("\nRobustness Summary (Top 10 most robust features):")
            display(robust_summary.nlargest(10, 'icc')[['feature_name', 'icc', 'cov_pct', 'robustness_label']])
            
            print("\nRobustness distribution:")
            print(robust_summary['robustness_label'].value_counts())
            
            # Store for visualization
            globals()['robustness_summary'] = robust_summary
            globals()['robust_features'] = robust_features
            
        except Exception as e:
            print(f"\n⚠️ Could not load robustness results: {e}")
            print("   Make sure you ran the robustness testing cell (6️⃣b)")

    # Load metadata
    try:
        meta_path = os.path.join(results_dir, "case_metadata.xlsx")
        metadata = pd.read_excel(meta_path)
        print(f"\n✅ Loaded metadata: {len(metadata)} courses")
        print("\nSummary:")
        print(f"  Patients: {metadata['PatientID'].nunique()}")
        print(f"  Courses: {len(metadata)}")
    except Exception as e:
        print(f"\n⚠️ Could not load metadata: {e}")

## 8️⃣ Quick Visualization

Create some basic plots:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

# DVH metrics visualization
try:
    # Plot mean dose by structure
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Mean dose
    top_structures = dvh.groupby('Structure')['Dmean_Gy'].mean().sort_values(ascending=False).head(10)
    top_structures.plot(kind='barh', ax=axes[0], color='steelblue')
    axes[0].set_xlabel('Mean Dose (Gy)')
    axes[0].set_title('Top 10 Structures by Mean Dose')

    # Volume distribution
    dvh['ROI_Volume_cc'].hist(bins=50, ax=axes[1], color='coral', edgecolor='black')
    axes[1].set_xlabel('ROI Volume (cc)')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('ROI Volume Distribution')
    axes[1].set_yscale('log')

    plt.tight_layout()
    plt.show()

    print("✅ Visualizations created")
except Exception as e:
    print(f"⚠️ Could not create visualizations: {e}")

### 🧪 Robustness Visualizations

Visualize feature stability metrics (only if robustness testing was enabled)

In [ ]:
if ENABLE_ROBUSTNESS and 'robustness_summary' in globals():
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    
    sns.set_style("whitegrid")
    
    try:
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. ICC Distribution
        ax = axes[0, 0]
        robustness_summary['icc'].hist(bins=50, ax=ax, color='steelblue', edgecolor='black', alpha=0.7)
        ax.axvline(0.90, color='green', linestyle='--', linewidth=2, label='Robust threshold (≥0.90)')
        ax.axvline(0.75, color='orange', linestyle='--', linewidth=2, label='Acceptable threshold (≥0.75)')
        ax.set_xlabel('ICC (Intraclass Correlation Coefficient)', fontsize=12)
        ax.set_ylabel('Number of Features', fontsize=12)
        ax.set_title('ICC Distribution Across All Features', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 2. CoV Distribution
        ax = axes[0, 1]
        # Filter out extreme values for better visualization
        cov_filtered = robustness_summary[robustness_summary['cov_pct'] <= 100]['cov_pct']
        cov_filtered.hist(bins=50, ax=ax, color='coral', edgecolor='black', alpha=0.7)
        ax.axvline(10, color='green', linestyle='--', linewidth=2, label='Robust threshold (≤10%)')
        ax.axvline(20, color='orange', linestyle='--', linewidth=2, label='Acceptable threshold (≤20%)')
        ax.set_xlabel('CoV - Coefficient of Variation (%)', fontsize=12)
        ax.set_ylabel('Number of Features', fontsize=12)
        ax.set_title('CoV Distribution Across All Features', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # 3. Robustness Category Distribution
        ax = axes[1, 0]
        category_counts = robustness_summary['robustness_label'].value_counts()
        colors = {'robust': 'green', 'acceptable': 'orange', 'poor': 'red'}
        category_colors = [colors.get(cat, 'gray') for cat in category_counts.index]
        category_counts.plot(kind='bar', ax=ax, color=category_colors, edgecolor='black', alpha=0.7)
        ax.set_xlabel('Robustness Category', fontsize=12)
        ax.set_ylabel('Number of Features', fontsize=12)
        ax.set_title('Feature Distribution by Robustness Category', fontsize=14, fontweight='bold')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add percentage labels on bars
        for i, (cat, count) in enumerate(category_counts.items()):
            pct = 100 * count / len(robustness_summary)
            ax.text(i, count + 1, f'{count}\n({pct:.1f}%)', ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        # 4. ICC vs CoV Scatter Plot
        ax = axes[1, 1]
        
        # Create color map for categories
        color_map = {'robust': 'green', 'acceptable': 'orange', 'poor': 'red'}
        colors_list = robustness_summary['robustness_label'].map(color_map)
        
        # Scatter plot (limit CoV to 100 for better visualization)
        plot_data = robustness_summary[robustness_summary['cov_pct'] <= 100]
        scatter = ax.scatter(plot_data['icc'], plot_data['cov_pct'], 
                           c=[color_map.get(x, 'gray') for x in plot_data['robustness_label']], 
                           alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
        
        # Add threshold lines
        ax.axvline(0.90, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='ICC ≥0.90')
        ax.axvline(0.75, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='ICC ≥0.75')
        ax.axhline(10, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='CoV ≤10%')
        ax.axhline(20, color='orange', linestyle='--', linewidth=1.5, alpha=0.7, label='CoV ≤20%')
        
        # Add shaded regions for robustness zones
        ax.fill_between([0.90, 1.0], 0, 10, alpha=0.1, color='green', label='Robust zone')
        ax.fill_between([0.75, 0.90], 0, 20, alpha=0.1, color='orange')
        
        ax.set_xlabel('ICC', fontsize=12)
        ax.set_ylabel('CoV (%)', fontsize=12)
        ax.set_title('ICC vs CoV: Feature Stability Landscape', fontsize=14, fontweight='bold')
        ax.set_xlim([0, 1.0])
        ax.set_ylim([0, 100])
        ax.legend(loc='upper right', fontsize=9)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print summary statistics
        print("\n" + "="*60)
        print("ROBUSTNESS SUMMARY STATISTICS")
        print("="*60)
        print(f"\nTotal features analyzed: {len(robustness_summary)}")
        print(f"\nICC Statistics:")
        print(f"  Mean ICC: {robustness_summary['icc'].mean():.3f}")
        print(f"  Median ICC: {robustness_summary['icc'].median():.3f}")
        print(f"  Features with ICC ≥0.90: {(robustness_summary['icc'] >= 0.90).sum()} ({100*(robustness_summary['icc'] >= 0.90).sum()/len(robustness_summary):.1f}%)")
        print(f"  Features with ICC ≥0.75: {(robustness_summary['icc'] >= 0.75).sum()} ({100*(robustness_summary['icc'] >= 0.75).sum()/len(robustness_summary):.1f}%)")
        
        print(f"\nCoV Statistics:")
        print(f"  Mean CoV: {robustness_summary['cov_pct'].mean():.2f}%")
        print(f"  Median CoV: {robustness_summary['cov_pct'].median():.2f}%")
        print(f"  Features with CoV ≤10%: {(robustness_summary['cov_pct'] <= 10).sum()} ({100*(robustness_summary['cov_pct'] <= 10).sum()/len(robustness_summary):.1f}%)")
        print(f"  Features with CoV ≤20%: {(robustness_summary['cov_pct'] <= 20).sum()} ({100*(robustness_summary['cov_pct'] <= 20).sum()/len(robustness_summary):.1f}%)")
        
        print(f"\nRobustness Categories:")
        for cat in ['robust', 'acceptable', 'poor']:
            count = (robustness_summary['robustness_label'] == cat).sum()
            pct = 100 * count / len(robustness_summary)
            print(f"  {cat.capitalize()}: {count} ({pct:.1f}%)")
        
        print("="*60)
        
        print("\n✅ Robustness visualizations created")
        
    except Exception as e:
        print(f"⚠️ Could not create robustness visualizations: {e}")
        import traceback
        traceback.print_exc()
        
elif ENABLE_ROBUSTNESS:
    print("⚠️ Robustness results not loaded. Make sure you ran the results viewing cell (7️⃣) first.")
else:
    print("ℹ️ Robustness testing was not enabled. Set ENABLE_ROBUSTNESS = True to generate robustness visualizations.")

## 9️⃣ Download Results

Download results to your local machine:

In [ ]:
%%bash
# Create ZIP archive of results
echo "Creating results archive..."
cd /content
zip -r -q results.zip output/_RESULTS/

echo "✅ Results archived: /content/results.zip"
ls -lh /content/results.zip

In [ ]:
from google.colab import files

print("Downloading results.zip...")
files.download('/content/results.zip')
print("\n✅ Download started. Check your browser's download folder.")

### Alternative: Save to Google Drive

If you mounted Google Drive earlier, copy results there:

In [ ]:
%%bash
# Check if Drive is mounted
if [ -d "/content/drive/MyDrive" ]; then
    echo "Copying results to Google Drive..."
    cp -r /content/output/_RESULTS /content/drive/MyDrive/rtpipeline_results_$(date +%Y%m%d_%H%M%S)
    echo "✅ Results copied to: /content/drive/MyDrive/rtpipeline_results_*"
else
    echo "⚠️ Google Drive not mounted. Run the 'Mount Google Drive' cell first."
fi

## 🧹 Cleanup (Optional)

Free up space by removing large intermediate files:

---

## 📚 Additional Resources

- **Output Format Guide:** [output_format.md](https://github.com/kstawiski/rtpipeline/blob/main/output_format.md)
- **Quick Reference:** [output_format_quick_ref.md](https://github.com/kstawiski/rtpipeline/blob/main/output_format_quick_ref.md)
- **Radiomics Robustness Guide:** [RADIOMICS_ROBUSTNESS.md](https://github.com/kstawiski/rtpipeline/blob/main/RADIOMICS_ROBUSTNESS.md)
- **GitHub Repository:** https://github.com/kstawiski/rtpipeline
- **Issues/Questions:** https://github.com/kstawiski/rtpipeline/issues

## ⚠️ Troubleshooting

**Pipeline fails with GPU errors:**
- Set `USE_GPU = False` in configuration cell
- Reduce `SEG_WORKERS` to 1

**Out of memory errors:**
- Reduce `WORKERS` and `SEG_WORKERS`
- Enable `FAST_MODE = True`
- Process patients in smaller batches
- For robustness testing, use "mild" intensity or reduce structures

**Colab timeout:**
- Upgrade to Colab Pro for longer runtime
- Process in batches
- Save intermediate results to Google Drive
- Run robustness testing separately after main pipeline

**Missing DICOM files:**
- Ensure DICOM directory is correct
- Check file permissions
- Verify .dcm file extensions

**Robustness testing fails:**
- Ensure main pipeline completed successfully first
- Check that radiomics extraction was enabled (`ENABLE_RADIOMICS = True`)
- Verify structures exist in segmentation outputs
- Try reducing `ROBUSTNESS_INTENSITY` to "mild"

---

**Notebook Version:** 2.0 (with Radiomics Robustness Testing)  
**Compatible with:** rtpipeline v2.0+ as of November 2025